## **1 - Importing Libraries & Utilities**

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import os
from IPython.display import clear_output
from time import sleep, perf_counter
from datetime import timedelta
from google.colab import files, drive
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

## **2 - Uploading Required Data Files**

In [ ]:
drive.mount('/content/drive')
path_files = '/content/drive/MyDrive/2023_DSMP_Project'

Mounted at /content/drive


In [ ]:
for file_name in sorted(os.listdir(path_files)):
  if '.xlsx' in file_name:
    print(file_name)

0 - Macro Indicators.xlsx
1 - Benchmark Prices.xlsx
2 - End - Use Prices.xlsx
3 - End - Use Demand.xlsx
4 - Primary Energy Demand.xlsx
5 - Electricity Generation.xlsx
6 - Electricity Capacity.xlsx
7 - Electricity Interchange.xlsx
8 - Weather_Data_Alberta_WU.xlsx
Alberta_EP_Monthly.xlsx
DSMP_Weather_Data_Canada.xlsx
Economics_Alberta.xlsx


### **2.1 - Joining Data Files - 2 & 3**

In [ ]:
for j in range(2,4):
  print(sorted(os.listdir(path_files))[j])
  df = pd.read_excel(path_files + '/' + sorted(os.listdir(path_files))[j])
  globals()[f"df_{j}"] = df
  del df

2 - End - Use Prices.xlsx
3 - End - Use Demand.xlsx


In [ ]:
# Joining df_2 and df_3
df_2_3 = df_2.merge(df_3, on = df_2.columns.tolist()[:4], how = 'outer')
del df_2, df_3
df_2_3.rename(columns = {'Price (2022 CAD $ per GJ)':'Price (CAD $/GJ)'}, inplace = True)
#
#df_0 = pd.read_excel(path_files + '/' +'0 - Macro Indicators.xlsx')
#df_2_3 = df_2_3.merge(df_0, on = ['Scenario', 'Year'], how = 'outer')
#df_2_3 = df2_3[(df_2_3['Value']!='Population (thousands)') & (df_2_3['Value']!='Population (thousands)')]
df_2_3

,Scenario,Sector,Region,Year,Price (CAD $/GJ),End-Use Demand (PetaJoules),GHG Emmisions (KTons_CO2_Equivalent)
0,Global Net-zero,Residential,Newfoundland and Labrador,2005,37.59,12.17,21.501038
1,Global Net-zero,Residential,Prince Edward Island,2005,45.86,0.58,9.204189
2,Global Net-zero,Residential,Nova Scotia,2005,41.17,14.86,21.941681
3,Global Net-zero,Residential,New Brunswick,2005,39.36,19.86,23.646266
4,Global Net-zero,Residential,Quebec,2005,25.46,206.31,18.720960
...,...,...,...,...,...,...,...
5377,Current Measures,Industrial,Alberta,2050,56.17,249.90,3.680523
5378,Current Measures,Industrial,British Columbia,2050,32.28,166.62,2.679146
5379,Current Measures,Industrial,Yukon,2050,57.38,0.11,0.298651
5380,Current Measures,Industrial,Northwest Territories,2050,41.94,0.08,0.324334


### **2.2 - Joining Data Files - 5 & 6**

In [ ]:
for j in range(5,7):
  print(sorted(os.listdir(path_files))[j])
  df = pd.read_excel(path_files + '/' + sorted(os.listdir(path_files))[j])
  globals()[f"df_{j}"] = df
  del df

5 - Electricity Generation.xlsx
6 - Electricity Capacity.xlsx


According to the [U.S. Energy Information Administration (EIA)](https://www.eia.gov/tools/faqs/faq.php?id=101&t=3):
- **Electricity Generation** is the amount of electricity a generator produces during a specific period of time.
- **Electricity Capacity** is the maximum electric output an electricity generator can produce under specific conditions.

In [ ]:
df_5_6 = df_5.merge(df_6, on = df_5.columns.tolist()[:4], how = 'outer')
del df_5, df_6
scenario_list = list(df_5_6['Scenario'].unique())
region_list = list(df_5_6['Region'].unique())
#df_5_6[(df_5_6['Scenario'] == 'Global Net-zero') & (df_5_6['Region'] == 'Newfoundland and Labrador')].head(15)

In [ ]:
sources_ = df_5_6['Sources'].unique().tolist()
#
df_5_6['Sources'] = df_5_6['Sources'].apply(lambda x: 'Renewable Energies_Biomass' if x in sources_[:4] else "Nuclear" if x == 'Uranium'	else "Refined Petroleum Products_Oil" if x == "Oil" else 'Coal, Coke and Coke Oven Gas' if x == 'Coal & Coke' else x)
#
df_5_6 = df_5_6.groupby(by = df_5_6.columns.tolist()[:4]).sum()
df_5_6 = df_5_6.reset_index().sort_values('Year')
#df_5_6[(df_5_6['Scenario'] == 'Global Net-zero') & (df_5_6['Region'] == 'Newfoundland and Labrador') & (df_5_6['Sources'] == 'Renewable Energies')]
#df_5_6
df_5_6["Scenario"] = pd.Categorical(df_5_6["Scenario"], categories = scenario_list)
#
df_5_6["Region"] = pd.Categorical(df_5_6["Region"], categories = region_list)
#
df_5_6.sort_values(by = ["Scenario","Region"])

,Scenario,Region,Sources,Year,Electricity Generation - (GWh),Electricity Capacity - (GWh)
6900,Global Net-zero,Newfoundland and Labrador,"Coal, Coke and Coke Oven Gas",2005,0.00,0.00
7084,Global Net-zero,Newfoundland and Labrador,Renewable Energies_Biomass,2005,40741.37,6805.68
7038,Global Net-zero,Newfoundland and Labrador,Refined Petroleum Products_Oil,2005,1306.21,783.70
6992,Global Net-zero,Newfoundland and Labrador,Nuclear,2005,0.00,0.00
6946,Global Net-zero,Newfoundland and Labrador,Natural Gas,2005,267.07,131.50
...,...,...,...,...,...,...
4829,Current Measures,Nunavut,Renewable Energies_Biomass,2050,332.81,102.89
4783,Current Measures,Nunavut,Refined Petroleum Products_Oil,2050,85.73,70.28
4737,Current Measures,Nunavut,Nuclear,2050,0.00,0.00
4691,Current Measures,Nunavut,Natural Gas,2050,0.00,0.00


### **2.3 - Joining Data Files - 4 and 5_6**

In [ ]:
j = 4
print(sorted(os.listdir(path_files))[j])
df_4 = pd.read_excel(path_files + '/' + sorted(os.listdir(path_files))[j])
df_4['Sources'] = df_4['Sources'].apply(lambda x: 'Renewable Energies_Biomass' if x in ['Hydro', 'Other Renewables and Landfill Gas'] else "Refined Petroleum Products_Oil" if x == 'Refined Petroleum Products' else x)
#df_4

4 - Primary Energy Demand.xlsx


In [ ]:
df_4 = df_4.groupby(by = df_4.columns.tolist()[:4]).sum()
df_4 = df_4.reset_index().sort_values('Year')
df_4["Scenario"] = pd.Categorical(df_4["Scenario"], categories = scenario_list)
#
df_4["Region"] = pd.Categorical(df_4["Region"], categories = region_list)
#
#df_4.sort_values(by = ["Scenario","Region"]).head(10)
#
# Joining df_4 and df_5_6
df_4 = df_4.merge(df_5_6, on = df_4.columns.tolist()[:4], how = 'outer')
del df_5_6
df_4["Scenario"] = pd.Categorical(df_4["Scenario"], categories = scenario_list)
#
df_4["Region"] = pd.Categorical(df_4["Region"], categories = region_list)
df_4

,Scenario,Region,Sources,Year,Primary Demand - Petajoules,Electricity Generation - (GWh),Electricity Capacity - (GWh)
0,Canada Net-zero,Alberta,"Coal, Coke and Coke Oven Gas",2005,547.86,43581.07,5696.00
1,Global Net-zero,Prince Edward Island,"Coal, Coke and Coke Oven Gas",2005,0.00,0.00,0.00
2,Current Measures,Nunavut,Natural Gas,2005,0.00,0.00,0.00
3,Global Net-zero,New Brunswick,Renewable Energies_Biomass,2005,64.22,4485.00,1080.50
4,Global Net-zero,Manitoba,"Coal, Coke and Coke Oven Gas",2005,2.79,413.26,97.64
...,...,...,...,...,...,...,...
8965,Canada Net-zero,Prince Edward Island,Nuclear,2050,31.45,3299.12,457.16
8966,Global Net-zero,Manitoba,Refined Petroleum Products_Oil,2050,37.90,3.79,37.31
8967,Canada Net-zero,Prince Edward Island,Refined Petroleum Products_Oil,2050,3.17,0.16,40.45
8968,Canada Net-zero,Yukon,Renewable Energies_Biomass,2050,3.03,605.95,225.24


### **2.4 - Joining Data Files - 4 and 7**

In [ ]:
j = 7
print(sorted(os.listdir(path_files))[j])
df_7 = pd.read_excel(path_files + '/' + sorted(os.listdir(path_files))[j])
# Let's delete some unuseful "Interchange Flows" categroies
Interchange_Flows = [flow for flow in df_7['Interchange Flows'].unique().tolist() if 'Net' not in flow]
df_7 = df_7[df_7['Interchange Flows'].isin(Interchange_Flows)]
# Joining df_4 and df_5_6
df_4 = df_4.merge(df_7, on = ['Scenario',	'Region',	'Year'], how = 'outer')
del df_7
#
df_4["Scenario"] = pd.Categorical(df_4["Scenario"], categories = scenario_list)
#
df_4["Region"] = pd.Categorical(df_4["Region"], categories = region_list)
df_4.sort_values(by = ["Scenario","Region"], inplace = True, ignore_index = True)
df_4

7 - Electricity Interchange.xlsx


,Scenario,Region,Sources,Year,Primary Demand - Petajoules,Electricity Generation - (GWh),Electricity Capacity - (GWh),Interchange Flows,Interchange (GWh)
0,Global Net-zero,Newfoundland and Labrador,"Coal, Coke and Coke Oven Gas",2005,3.11,0.00,0.00,Interprovincial In-Flows,15.69
1,Global Net-zero,Newfoundland and Labrador,"Coal, Coke and Coke Oven Gas",2005,3.11,0.00,0.00,Interprovincial Out-Flows,30204.72
2,Global Net-zero,Newfoundland and Labrador,"Coal, Coke and Coke Oven Gas",2005,3.11,0.00,0.00,Exports,0.00
3,Global Net-zero,Newfoundland and Labrador,"Coal, Coke and Coke Oven Gas",2005,3.11,0.00,0.00,Imports,0.00
4,Global Net-zero,Newfoundland and Labrador,Renewable Energies_Biomass,2005,162.02,40741.37,6805.68,Interprovincial In-Flows,15.69
...,...,...,...,...,...,...,...,...,...
35875,Current Measures,Nunavut,Natural Gas,2050,0.00,0.00,0.00,Imports,0.00
35876,Current Measures,Nunavut,"Coal, Coke and Coke Oven Gas",2050,0.00,0.00,0.00,Interprovincial In-Flows,0.00
35877,Current Measures,Nunavut,"Coal, Coke and Coke Oven Gas",2050,0.00,0.00,0.00,Interprovincial Out-Flows,0.00
35878,Current Measures,Nunavut,"Coal, Coke and Coke Oven Gas",2050,0.00,0.00,0.00,Exports,0.00


According to the **Michigan Public Service Comminsion** (**MSPC**):

“**Inflow**” means the electricity a customer uses from the utility distribution system. “**Outflow**” means the electricity
generated by the customer’s distributed generation project that is not used on-site and is instead sent to the electric
grid.

**Courtesy:**

https://www.michigan.gov/-/media/Project/Websites/mpsc/consumer/info/briefs/Distributed_Generation_Issue_Brief_4-18-18.pdf?rev=3bcf2592c5a64f328220811750e35496#:~:text=%E2%80%9CInflow%E2%80%9D%20means%20the%20electricity%20a,sent%20to%20the%20electric%20grid.

### **2.5 - Joining Data Files - 2_3 and 4 DataFrames**

In [ ]:
df = df_4.merge(df_2_3, on = ['Scenario',	'Region',	'Year'], how = 'outer')
cols = df.columns.tolist()
new_cols = cols[:-3] + cols[-2:] + [cols[-3]]
df = df[new_cols]
df.head(5)

,Scenario,Region,Sources,Year,Primary Demand - Petajoules,Electricity Generation - (GWh),Electricity Capacity - (GWh),Interchange Flows,Interchange (GWh),Sector,End-Use Demand (PetaJoules),GHG Emmisions (KTons_CO2_Equivalent),Price (CAD $/GJ)
0,Global Net-zero,Newfoundland and Labrador,"Coal, Coke and Coke Oven Gas",2005,3.11,0.0,0.0,Interprovincial In-Flows,15.69,Residential,12.17,21.501038,37.59
1,Global Net-zero,Newfoundland and Labrador,"Coal, Coke and Coke Oven Gas",2005,3.11,0.0,0.0,Interprovincial In-Flows,15.69,Commercial,7.72,13.639113,29.12
2,Global Net-zero,Newfoundland and Labrador,"Coal, Coke and Coke Oven Gas",2005,3.11,0.0,0.0,Interprovincial In-Flows,15.69,Industrial,15.68,27.702241,21.27
3,Global Net-zero,Newfoundland and Labrador,"Coal, Coke and Coke Oven Gas",2005,3.11,0.0,0.0,Interprovincial Out-Flows,30204.72,Residential,12.17,21.501038,37.59
4,Global Net-zero,Newfoundland and Labrador,"Coal, Coke and Coke Oven Gas",2005,3.11,0.0,0.0,Interprovincial Out-Flows,30204.72,Commercial,7.72,13.639113,29.12


### **2.6 - Post-Processing**

In [ ]:
df.loc[df['Sources']=='Renewable Energies_Biomass', 'GHG Emmisions (KTons_CO2_Equivalent)'] = 0
#
df.loc[(df['Sources']=='Nuclear'), 'GHG Emmisions (KTons_CO2_Equivalent)'] = 0
df.loc[(df['Sources']=='Nuclear') & (df['Electricity Capacity - (GWh)']==0), 'Interchange (GWh)'] = 0
df.loc[(df['Sources']=='Nuclear') & (df['Electricity Capacity - (GWh)']==0), 'Price (CAD $/GJ)'] = 0

## **3 - Integrating Weather Data**

### **3.1 - Subsetting Alberta's DataFrame**

In [ ]:
data = df[df['Region'] == 'Alberta'].reset_index(drop = True)
data.drop(['Region'], axis = 1, inplace = True)
data.head(5)

,Scenario,Sources,Year,Primary Demand - Petajoules,Electricity Generation - (GWh),Electricity Capacity - (GWh),Interchange Flows,Interchange (GWh),Sector,End-Use Demand (PetaJoules),GHG Emmisions (KTons_CO2_Equivalent),Price (CAD $/GJ)
0,Global Net-zero,"Coal, Coke and Coke Oven Gas",2005,547.86,43581.07,5696.0,Interprovincial In-Flows,1740.06,Residential,27.97,8.318513,33.21
1,Global Net-zero,"Coal, Coke and Coke Oven Gas",2005,547.86,43581.07,5696.0,Interprovincial In-Flows,1740.06,Commercial,45.73,13.600485,26.44
2,Global Net-zero,"Coal, Coke and Coke Oven Gas",2005,547.86,43581.07,5696.0,Interprovincial In-Flows,1740.06,Industrial,137.60,40.923394,21.92
3,Global Net-zero,"Coal, Coke and Coke Oven Gas",2005,547.86,43581.07,5696.0,Interprovincial Out-Flows,1198.17,Residential,27.97,8.318513,33.21
4,Global Net-zero,"Coal, Coke and Coke Oven Gas",2005,547.86,43581.07,5696.0,Interprovincial Out-Flows,1198.17,Commercial,45.73,13.600485,26.44


### **3.2 - Integrating MacroEconomic Data**

In [ ]:
macro_econom = pd.read_excel(path_files + '/' + 'Economics_Alberta.xlsx', sheet_name='Macro_Econom')
macro_econom = macro_econom.iloc[:,:-1]
#macro_econom['Time'] = pd.to_datetime(macro_econom['Time']).dt.to_period('D')
#macro_econom.head(5)
data = data.merge(macro_econom.iloc[:,:-1], on = 'Year', how = 'outer')
#
last_column = data.pop('Price (CAD $/GJ)')
data.insert(data.shape[1], 'Price (CAD $/GJ)', last_column)
#data

In [ ]:
data['Year'] = pd.to_datetime(data['Year'], format = '%Y')
data.rename(columns = {'Year':'Time'}, inplace = True)
#
data['Time'] = data['Time'].apply(lambda x: x - np.timedelta64(1, 'M')) #'M'
data['Time'] = data['Time'].dt.to_period('D') # 'M'
#
data = data[data['Time']<'2024-01-01']
data.reset_index(inplace = True, drop = True)
#data

data.to_excel('data.xlsx', index = False, freeze_panes = (1,0))
files.download('data.xlsx')

### **3.3 - Uploading Alberta's Weather Data**

In [ ]:
wu_data = pd.read_excel(path_files + '/' + '8 - Weather_Data_Alberta_WU.xlsx')
print(sorted(os.listdir(path_files))[-4])
#
wu = wu_data#.groupby(pd.Grouper(key = 'Time', axis = 0, freq = 'M')).mean() # ======> Delete "#" at "#.groupby" when grouping on a monthly basis
#wu.reset_index(inplace = True)          # ======> # Activate this when working on a monthly basis
#wu['Time'] = wu['Time'].dt.to_period('M')         # Activate this when working on a monthly basis
wu = wu[wu['Time']>= '2004-12-31']               # Deactivate this when working on a monthly basis
wu.reset_index(inplace = True, drop = True)      #       "      "    "     "     " "    "      "
wu.drop(['Region'], axis = 1, inplace = True)    #       "      "    "     "     " "    "      "
wu['Time'] = wu['Time'].dt.to_period('D')        #       "      "    "     "     " "    "      "
wu

DSMP_Weather_Data_Canada.xlsx


,Time,Temp_Max (°F),Temp_Avg (°F),Temp_Min (°F),Hum_Max (%),Hum_Avg (%),Hum_Min (%),Wind_Speed_Max (mph),Wind_Speed_Avg (mph),Wind_Speed_Min (mph)
0,2004-12-31,0,-8.9,-18,77,58.5,0,12,4.6,0
1,2005-01-01,0,-3.7,-11,77,58.5,0,9,5.0,0
2,2005-01-02,0,-13.3,-24,77,61.9,0,7,4.0,2
3,2005-01-03,0,-8.6,-26,91,51.7,0,7,2.5,0
4,2005-01-04,3,-7.5,-20,85,69.2,0,16,8.6,0
...,...,...,...,...,...,...,...,...,...,...
6935,2023-12-27,36,24.0,12,80,68.7,52,16,8.9,0
6936,2023-12-28,46,29.6,18,86,71.7,49,10,5.5,0
6937,2023-12-29,37,25.2,14,93,76.0,60,12,5.3,0
6938,2023-12-30,37,29.0,23,93,78.8,65,18,11.0,6


### **3.4 - Expanding Alberta's Dataset - Monthly Basis**

In [ ]:
sample = data[data['Time']<='2004-12-31'].select_dtypes(include='O')
sample.reset_index(drop = True, inplace = True)

In [ ]:
temp = pd.DataFrame()
for time in wu['Time']:
  print(time)
  temp_1 = pd.concat([pd.DataFrame([time]*sample.shape[0], columns = ['Time']), sample], axis = 1)
  #
  temp = pd.concat([temp, temp_1], axis = 0, ignore_index = True)
  del temp_1
  clear_output()
del sample

In [ ]:
temp = pd.merge(temp, wu, left_on = 'Time', right_on = 'Time')
print('temp_shape : ', temp.shape)
#temp

temp_shape :  (1249560, 14)


In [ ]:
data = data.merge(temp, on = ['Time', 'Scenario', 'Sources', 'Interchange Flows', 'Sector'], how = 'outer')
print('data_shape : ', data.shape)
# Shift column 'Price (CAD $/GJ)' to first position
last_column = data.pop('Price (CAD $/GJ)')
# Insert column using insert(position,column_name,first_column) function
data.insert(data.shape[1], 'Price (CAD $/GJ)', last_column)
#
#data = data[data['Time']>='2018-12-31']
#print('data_shape : ', data.shape)
#data.reset_index(inplace = True, drop = True)
#
data.head(5)

data_shape :  (1249740, 23)


,Scenario,Sources,Time,Primary Demand - Petajoules,Electricity Generation - (GWh),Electricity Capacity - (GWh),Interchange Flows,Interchange (GWh),Sector,End-Use Demand (PetaJoules),...,Temp_Max (°F),Temp_Avg (°F),Temp_Min (°F),Hum_Max (%),Hum_Avg (%),Hum_Min (%),Wind_Speed_Max (mph),Wind_Speed_Avg (mph),Wind_Speed_Min (mph),Price (CAD $/GJ)
0,Global Net-zero,"Coal, Coke and Coke Oven Gas",2004-12-01,547.86,43581.07,5696.0,Interprovincial In-Flows,1740.06,Residential,27.97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.21
1,Global Net-zero,"Coal, Coke and Coke Oven Gas",2004-12-01,547.86,43581.07,5696.0,Interprovincial In-Flows,1740.06,Commercial,45.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.44
2,Global Net-zero,"Coal, Coke and Coke Oven Gas",2004-12-01,547.86,43581.07,5696.0,Interprovincial In-Flows,1740.06,Industrial,137.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.92
3,Global Net-zero,"Coal, Coke and Coke Oven Gas",2004-12-01,547.86,43581.07,5696.0,Interprovincial Out-Flows,1198.17,Residential,27.97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.21
4,Global Net-zero,"Coal, Coke and Coke Oven Gas",2004-12-01,547.86,43581.07,5696.0,Interprovincial Out-Flows,1198.17,Commercial,45.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.44


In [ ]:
data.isnull().sum()

Scenario                                              0
Sources                                               0
Time                                                  0
Primary Demand - Petajoules                     1246140
Electricity Generation - (GWh)                  1246140
Electricity Capacity - (GWh)                    1246140
Interchange Flows                                     0
Interchange (GWh)                               1246140
Sector                                                0
End-Use Demand (PetaJoules)                     1246140
GHG Emmisions (KTons_CO2_Equivalent)            1246140
Population                                      1246140
Real Gross Domestic Product ($2012 Millions)    1246140
Temp_Max (°F)                                       180
Temp_Avg (°F)                                       180
Temp_Min (°F)                                       180
Hum_Max (%)                                         180
Hum_Avg (%)                                     

## **4 - Filling Missing Values - Daily Basis**

#### **5.1 - Applying KNN Imputer**

In [ ]:
num_cols = data.select_dtypes(include = [np.number, 'period[D]','float64']).columns.tolist()
data_num = data[num_cols]

In [ ]:
#num_cols

from sklearn.impute import KNNImputer
mae_ = []
for i in range(1,40):            # i: k_value for nearest-neighbors
    print('i =', i)
    imputer = KNNImputer(n_neighbors = i, weights='uniform', metric='nan_euclidean')
    imputer.fit(data_num.iloc[:,1:])
    temp_ = imputer.transform(data_num.iloc[:,1:])
    temp_ = pd.DataFrame(temp_)
    mae_.append(abs(data.iloc[:,-1].std(axis=0, skipna=True)- temp_.iloc[:,-1].std(axis=0, skipna=True)))
    clear_output()

plt.figure(figsize = (15,5))
plt.plot([i for i in range(1,len(mae_)+1)], mae_, color='blue', linestyle='dashed',
         marker='o', markerfacecolor='red', markersize=10)
Naming the x-axis, y-axis and the whole graph
plt.xlabel("k_value")
plt.ylabel('MAE')
plt.show()

In [ ]:
from sklearn.impute import KNNImputer
# define imputer
imputer = KNNImputer(n_neighbors = 4, weights='uniform', metric='nan_euclidean')

In [ ]:
new_df = pd.DataFrame()
for scenario in data['Scenario'].unique():
    for source in data['Sources'].unique():
        for sector in data['Sector'].unique():
            for flow in data['Interchange Flows'].unique():
                df_ = data[(data['Scenario'] == scenario) & (data['Sources'] == source) & (data['Sector'] == sector) & (data['Interchange Flows'] == flow)]
                data_num = df_[num_cols]
                imputer.fit(data_num.iloc[:,1:])
                # transform the dataset
                data_num.iloc[:,1:] = imputer.transform(data_num.iloc[:,1:])
                data_num = pd.DataFrame(data_num, columns = num_cols)
                for col in num_cols:
                    df_[col] = data_num[col]
                #
                new_df = pd.concat([new_df, df_], axis = 0, ignore_index = True)

In [ ]:
new_df.isnull().sum()

Scenario                                        0
Sources                                         0
Time                                            0
Primary Demand - Petajoules                     0
Electricity Generation - (GWh)                  0
Electricity Capacity - (GWh)                    0
Interchange Flows                               0
Interchange (GWh)                               0
Sector                                          0
End-Use Demand (PetaJoules)                     0
GHG Emmisions (KTons_CO2_Equivalent)            0
Population                                      0
Real Gross Domestic Product ($2012 Millions)    0
Temp_Max (°F)                                   0
Temp_Avg (°F)                                   0
Temp_Min (°F)                                   0
Hum_Max (%)                                     0
Hum_Avg (%)                                     0
Hum_Min (%)                                     0
Wind_Speed_Max (mph)                            0


#### **5.2 - Visualizations**

In [ ]:
data.sort_values(by = 'Time', axis = 0, ascending = True, inplace = True)
new_df.sort_values(by = 'Time', axis = 0, ascending = True, inplace = True)

In [ ]:
vv

NameError: name 'ss' is not defined

In [ ]:
for source in data['Sources'].unique():
    for flow in data['Interchange Flows'].unique():
        for sector in data['Sector'].unique():
          df_ = new_df[(new_df['Sector'] == sector) & (new_df['Scenario'] == 'Current Measures') & (new_df['Sources'] == source) & (new_df['Interchange Flows'] == flow)]
          plt.figure(figsize=(15,5))
          plt.plot(pd.to_datetime(data['Time'].astype(str)), data['Price (CAD $/GJ)'], color='b', label = 'Raw')
          plt.plot(pd.to_datetime(df_['Time'].astype(str)), df_['Price (CAD $/GJ)'], color='k', linestyle = 'dashed', label = 'Imputed')
          # Naming the x-axis, y-axis and the whole graph
          plt.xlabel("timestamp")
          plt.ylabel('Price (CAD $/GJ)')
          plt.title(source + ' - ' + flow + ' - ' + sector)
          #
          plt.legend()
          plt.show()
          print('\n')

In [ ]:
new_df[new_df['Time']>='2014-01-01'].shape

(657180, 23)

In [ ]:
new_df[new_df['Time']>='2014-01-01'].to_excel('Alberta_EP_Daily_Macro_Econ_.xlsx', index = False, freeze_panes=(1,0))
files.download('Alberta_EP_Daily_Macro_Econ_.xlsx')